In [2]:
import lightgbm as lgbm
from scipy import sparse as ssp
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [3]:
def Gini(y_true, y_pred):
    # check and get number of samples
    assert y_true.shape == y_pred.shape
    n_samples = y_true.shape[0]

    # sort rows on prediction column
    # (from largest to smallest)
    arr = np.array([y_true, y_pred]).transpose()
    true_order = arr[arr[:, 0].argsort()][::-1, 0]
    pred_order = arr[arr[:, 1].argsort()][::-1, 0]

    # get Lorenz curves
    L_true = np.cumsum(true_order) * 1. / np.sum(true_order)
    L_pred = np.cumsum(pred_order) * 1. / np.sum(pred_order)
    L_ones = np.linspace(1 / n_samples, 1, n_samples)

    # get Gini coefficients (area between curves)
    G_true = np.sum(L_ones - L_true)
    G_pred = np.sum(L_ones - L_pred)

    # normalize to true Gini coefficient
    return G_pred * 1. / G_true


In [4]:
cv_only = True
save_cv = True
full_train = False

def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    return 'gini', Gini(labels, preds), True

In [5]:
path = "D:/Github/machine learning/Kaggle/Safe Driver Prediction/"

In [6]:
train = pd.read_csv(path+'train.csv')
train_label = train['target']
train_id = train['id']
test = pd.read_csv(path+'test.csv')
test_id = test['id']

In [7]:
NFOLDS = 5
kfold = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=218)

In [8]:
y = train['target'].values
drop_feature = [
    'id',
    'target'
]

X = train.drop(drop_feature,axis=1)

In [9]:
train.head()

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,0,2,2,5,1,0,0,1,0,...,9,1,5,8,0,1,1,0,0,1
1,9,0,1,1,7,0,0,0,0,1,...,3,1,1,9,0,1,1,0,1,0
2,13,0,5,4,9,1,0,0,0,1,...,4,2,7,7,0,1,1,0,1,0
3,16,0,0,1,2,0,0,1,0,0,...,2,2,4,9,0,0,0,0,0,0
4,17,0,0,2,0,1,0,1,0,0,...,3,1,1,3,0,0,0,1,1,0


In [10]:
feature_names = X.columns.tolist()
cat_features = [c for c in feature_names if ('cat' in c and 'count' not in c)]
num_features = [c for c in feature_names if ('cat' not in c and 'calc' not in c)]

In [11]:
train['missing'] = (train==-1).sum(axis=1).astype(float)
test['missing'] = (test==-1).sum(axis=1).astype(float)
num_features.append('missing')

In [12]:
for c in cat_features:
    le = LabelEncoder()
    le.fit(train[c])
    train[c] = le.transform(train[c])
    test[c] = le.transform(test[c])

enc = OneHotEncoder()
enc.fit(train[cat_features])
X_cat = enc.transform(train[cat_features])
X_t_cat = enc.transform(test[cat_features])

In [13]:
ind_features = [c for c in feature_names if 'ind' in c]
count=0
for c in ind_features:
    if count==0:
        train['new_ind'] = train[c].astype(str)+'_'
        test['new_ind'] = test[c].astype(str)+'_'
        count+=1
    else:
        train['new_ind'] += train[c].astype(str)+'_'
        test['new_ind'] += test[c].astype(str)+'_'

In [14]:
cat_count_features = []
for c in cat_features+['new_ind']:
    d = pd.concat([train[c],test[c]]).value_counts().to_dict()
    train['%s_count'%c] = train[c].apply(lambda x:d.get(x,0))
    test['%s_count'%c] = test[c].apply(lambda x:d.get(x,0))
    cat_count_features.append('%s_count'%c)

In [15]:
train_list = [train[num_features+cat_count_features].values,X_cat,]
test_list = [test[num_features+cat_count_features].values,X_t_cat,]


In [36]:
X = ssp.hstack(train_list).tocsr()
X_test = ssp.hstack(test_list).tocsr()

In [16]:
learning_rate = 0.1
num_leaves = 15
min_data_in_leaf = 2000
feature_fraction = 0.6
num_boost_round = 10000
params = {"objective": "binary",
          "boosting_type": "gbdt",
          "learning_rate": learning_rate,
          "num_leaves": num_leaves,
           "max_bin": 256,
          "feature_fraction": feature_fraction,
          "verbosity": 0,
          "drop_rate": 0.1,
          "is_unbalance": False,
          "max_drop": 50,
          "min_child_samples": 10,
          "min_child_weight": 150,
          "min_split_gain": 0,
          "subsample": 0.9
          }


In [37]:
x_score = []
final_cv_train = np.zeros(len(train_label))
final_cv_pred = np.zeros(len(test_id))
for s in range(16):
    cv_train = np.zeros(len(train_label))
    cv_pred = np.zeros(len(test_id))

    params['seed'] = s
    
    if cv_only:
        kf = kfold.split(X, train_label)

        best_trees = []
        fold_scores = []

        for i, (train_fold, validate) in enumerate(kf):
            X_train, X_validate, label_train, label_validate = \
                X[train_fold,:], X[validate,:], train_label[train_fold], train_label[validate]
            dtrain = lgbm.Dataset(X_train, label_train)
            dvalid = lgbm.Dataset(X_validate, label_validate, reference=dtrain)
            bst = lgbm.train(params, dtrain, num_boost_round, valid_sets=dvalid, feval=evalerror, verbose_eval=100,
                            early_stopping_rounds=100)
            best_trees.append(bst.best_iteration)
            cv_pred += bst.predict(X_test, num_iteration=bst.best_iteration)
            cv_train[validate] += bst.predict(X_validate)

            score = Gini(label_validate, cv_train[validate])
            print (score)
            fold_scores.append(score)

        cv_pred /= NFOLDS
        final_cv_train += cv_train
        final_cv_pred += cv_pred

        print("cv score:")
        print (Gini(train_label, cv_train))
        print ("current score:", Gini(train_label, final_cv_train / (s + 1.)), s+1)
        print(fold_scores)
        print(best_trees, np.mean(best_trees))

        x_score.append(Gini(train_label, cv_train))

print(x_score)
pd.DataFrame({'id': test_id, 'target': final_cv_pred / 16.}).to_csv(path+'pred_avg.csv', index=False)
pd.DataFrame({'id': train_id, 'target': final_cv_train / 16.}).to_csv(path+'cv_avg.csv', index=False)


Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.151401	valid_0's gini: 0.293401
[200]	valid_0's binary_logloss: 0.151285	valid_0's gini: 0.296137
[300]	valid_0's binary_logloss: 0.151296	valid_0's gini: 0.295778
Early stopping, best iteration is:
[235]	valid_0's binary_logloss: 0.151265	valid_0's gini: 0.296665
0.29666456778791145
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.15216	valid_0's gini: 0.271447
[200]	valid_0's binary_logloss: 0.152144	valid_0's gini: 0.27349
Early stopping, best iteration is:
[145]	valid_0's binary_logloss: 0.152127	valid_0's gini: 0.272928
0.2729277334928809
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.152081	valid_0's gini: 0.279382
[200]	valid_0's binary_logloss: 0.152048	valid_0's gini: 0.28106
Early stopping, best iteration is:
[129]	valid_0's binary_logloss: 0.152014	valid_0's gini: 0.281321
0.28132063

[300]	valid_0's binary_logloss: 0.152184	valid_0's gini: 0.274817
Early stopping, best iteration is:
[205]	valid_0's binary_logloss: 0.152146	valid_0's gini: 0.274895
0.27489529368834104
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.15207	valid_0's gini: 0.279475
[200]	valid_0's binary_logloss: 0.152035	valid_0's gini: 0.281024
[300]	valid_0's binary_logloss: 0.152056	valid_0's gini: 0.281188
Early stopping, best iteration is:
[270]	valid_0's binary_logloss: 0.152017	valid_0's gini: 0.28151
0.28150996851408605
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.151784	valid_0's gini: 0.280768
[200]	valid_0's binary_logloss: 0.151752	valid_0's gini: 0.282347
Early stopping, best iteration is:
[147]	valid_0's binary_logloss: 0.151718	valid_0's gini: 0.282987
0.28298713530564457
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.151477	valid_0's gin

[200]	valid_0's binary_logloss: 0.151732	valid_0's gini: 0.28439
[300]	valid_0's binary_logloss: 0.151781	valid_0's gini: 0.284739
Early stopping, best iteration is:
[201]	valid_0's binary_logloss: 0.151729	valid_0's gini: 0.284521
0.2845214444293345
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.151476	valid_0's gini: 0.294332
[200]	valid_0's binary_logloss: 0.151476	valid_0's gini: 0.294753
Early stopping, best iteration is:
[155]	valid_0's binary_logloss: 0.15143	valid_0's gini: 0.295816
0.29581611108323624
cv score:
0.2871546368219128
current score: 0.2893751595015699 9
[0.2970224214452311, 0.2757185596315549, 0.2829690945796141, 0.2845214444293345, 0.29581611108323624]
[217, 107, 178, 201, 155] 171.6
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.151472	valid_0's gini: 0.292536
[200]	valid_0's binary_logloss: 0.151406	valid_0's gini: 0.293324
[300]	valid_0's binary_logloss: 0.151443

[100]	valid_0's binary_logloss: 0.151422	valid_0's gini: 0.294291
[200]	valid_0's binary_logloss: 0.15135	valid_0's gini: 0.294928
Early stopping, best iteration is:
[135]	valid_0's binary_logloss: 0.151366	valid_0's gini: 0.295136
0.2951355322304417
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.152162	valid_0's gini: 0.272096
[200]	valid_0's binary_logloss: 0.152211	valid_0's gini: 0.272337
Early stopping, best iteration is:
[120]	valid_0's binary_logloss: 0.152155	valid_0's gini: 0.272699
0.2726994717885202
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's binary_logloss: 0.151981	valid_0's gini: 0.282198
[200]	valid_0's binary_logloss: 0.151905	valid_0's gini: 0.284303
[300]	valid_0's binary_logloss: 0.151991	valid_0's gini: 0.281771
Early stopping, best iteration is:
[209]	valid_0's binary_logloss: 0.151883	valid_0's gini: 0.285035
0.28503507136225276
Training until validation scores don't improve for 1